In [1]:
import unittest
import requests
import pandas as pd
import numpy as np
import os.path
from FIFAApp import *
from FIFADataWebscrapper import *

class Fifatestcase(unittest.TestCase):
    
    # Test request to site is successful
    def test_webrequest(self):
        response = requests.get('https://www.futbin.com/20/players?page=1&sort=Player_Rating&order=desc&version=gold')
        self.assertEqual(response.status_code, 200)
        
    # Test if csv file was written
    def test_webscrapper(self):
        self.assertTrue(os.path.exists('FIFA Player Info.csv'))
    
    # Test the structure of csv file
    def test_fifastructure(self):
        df = pd.read_csv('FIFA Player Info.csv', keep_default_na = False) 
        self.assertEqual(df.dtypes.tolist(), df.dtypes.tolist())
    
    # Test the structure of csv file    
    def test_fifastructure2(self):
        df = pd.read_csv('FIFA Player Info.csv', keep_default_na = False) 
        self.assertEqual(len(df.columns), 21)
        self.assertEqual(len(df), 16359)
    
    # Test position group support function
    def test_positiongroup(self):
        df = pd.read_csv('FIFA Player Info.csv', keep_default_na = False) 
        df['Position'].apply(position)
        self.assertListEqual(df['Position Group'].unique().tolist(), ['Attacker', 'Midfielder', 'Goal Keeper', 'Defender'])
    
    # Test continent code support function
    def test_continentcode(self):
        df = pd.read_csv('FIFA Player Info.csv', keep_default_na = False) 
        df['Country'].head(100).apply(continent)
        self.assertEqual(df['Continent'].isna().sum(), 0)
    
    # Test for duplicates
    def test_duplicates(self):
        df = pd.read_csv('FIFA Player Info.csv', keep_default_na = False)
        dup = df.duplicated(subset = ['Name', 'Overall Rating', 'Position'])
        self.assertEqual(dup.value_counts().tolist()[0], 16359)
    
    # Test select function for scatter plot
    def test_select_scatter(self):
        df = pd.read_csv('FIFA Player Info.csv', keep_default_na = False)
        overall_rating_val = [df['Overall Rating'].min(), df['Overall Rating'].max()]
        skill_level = [df['Skill'].min(), df['Skill'].max()]
        self.assertEqual(len(select_fifa()), 16359)
    
    # Test select histogram function
    def test_select_histogram(self):
        df = pd.read_csv('FIFA Player Info.csv', keep_default_na = False)
        selected, skill = select_hist()
        self.assertEqual(skill, "Overall Rating")
        
    # Test select best starting eleven
    def test_select_best(self):
        df = pd.read_csv('FIFA Player Info.csv', keep_default_na = False)
        selected = select_fifa2()
        self.assertEqual(len(selected), 11)
    
    # Test select best starting eleven contains Lionel Messi
    def test_select_best2(self):
        df = pd.read_csv('FIFA Player Info.csv', keep_default_na = False)
        selected = select_fifa2()
        messi = selected[selected.Name.str.contains('Lionel Messi')==True]
        self.assertEqual(messi.Name[0], 'Lionel Messi')
    
    # Test select number of defenders in best starting eleven 
    def test_select_best3(self):
        df = pd.read_csv('FIFA Player Info.csv', keep_default_na = False)
        selected = select_fifa2()
        num_def = selected[selected['Position Group'] == 'Defender']
        self.assertEqual(len(num_def), 4)
    
    # Test if only one goalie is picked
    def test_bestplayers_singleGoalie(self):
        df = pd.read_csv('FIFA Player Info.csv', keep_default_na = False)
        best = bestplayers(df, '442', 'Overall Rating')
        goalie_num = len(best[best['Position Group'] == 'Goal Keeper'])
        self.assertEqual(goalie_num, 1)
    
    # Test if number of defenders are equal to formation selected
    def test_bestplayers_defenders(self):
        df = pd.read_csv('FIFA Player Info.csv', keep_default_na = False)
        best = bestplayers(df, '532', 'Defending')
        def_num = len(best[best['Position Group'] == 'Defender'])
        self.assertEqual(def_num, 5)
        
    # Test if number of midfielders are equal to formation selected
    def test_bestplayers_midfielder(self):
        df = pd.read_csv('FIFA Player Info.csv', keep_default_na = False)
        best = bestplayers(df, '442', 'Overall Rating')
        mf_num = len(best[best['Position Group'] == 'Midfielder'])
        self.assertEqual(mf_num, 4)
    
    # Test if number of attackers are equal to formation selected
    def test_bestplayers_attacker(self):
        df = pd.read_csv('FIFA Player Info.csv', keep_default_na = False)
        best = bestplayers(df, '442', 'Shooting')
        atk_num = len(best[best['Position Group'] == 'Attacker'])
        self.assertNotEqual(atk_num, 3)
    
    # Test if any formation yeilds 433 formation
    def test_bestformation(self):
        df = pd.read_csv('FIFA Player Info.csv', keep_default_na = False)
        best = bestformation(df, 'Any', 'Overall Rating')
        atk_num = len(best[best['Position Group'] == 'Attacker'])
        mid_num = len(best[best['Position Group'] == 'Midfielder'])
        def_num = len(best[best['Position Group'] == 'Defender'])
        self.assertEqual(atk_num, 3)
        self.assertEqual(mid_num, 3)
        self.assertEqual(def_num, 4)
    
    def test_is_csv_read(self):
        # tests that CSV file read by checking player at index 1
        fifa = pd.read_csv('FIFA Player Info.csv', keep_default_na = False)
        fifa.columns = fifa.columns.str.strip().str.lower().\
        str.replace(' ', '_').str.replace('(', '').str.replace(')', '')
        overall_rating_val = fifa.overall_rating[1]
        skill_level = fifa.skill[1]
        player_val = fifa.name[1]
        league_val = fifa.league[1]
        club_val = fifa.club[1]
        country_val = fifa.country[1]
        position_val = fifa.position[1]        
        
        
        self.assertEqual(overall_rating_val, 93)
        self.assertEqual(skill_level, 5)
        self.assertEqual(player_val, "Cristiano Ronaldo")
        self.assertEqual(league_val, "Serie A TIM")
        self.assertEqual(club_val, "Piemonte Calcio")
        self.assertEqual(country_val, "Portugal")
        self.assertEqual(position_val, "ST")    
    
    def test_select_fifa_check(self):
        # tests that player data exists within Overall Rating
        # and Skill range shown before checking select_fifa

        selected = fifa[
        (fifa['Overall Rating'] >= 0) &
        (fifa['Overall Rating'] <= 100) & # all players by overall rating
        (fifa['Skill'] >= 0) &
        (fifa['Skill'] <= 5) # all players by skill
        ]      # ensures all players in data set
        self.assertEqual(selected.Name[0],"Lionel Messi")
        self.assertEqual(selected.League[0],"LaLiga Santander")
        self.assertEqual(selected.Club[0],"FC Barcelona")
        self.assertEqual(selected.Country[0],"Argentina")
        self.assertEqual(selected.Position[0],"RW")
        
        # checks that select_fifa function returns the proper color
        # and percentile given user input (here, check to ensure
        # color and percentile definition agree relative to data set)
        self.assertEqual(select_fifa().color[int(len(fifa)/2)],"#FD9F6C")
        self.assertEqual(select_fifa().legend[int(len(fifa)/2)],"50th Percentile")
        
        self.assertEqual(select_fifa().color[int(len(fifa)/6)],"#29788E")
        self.assertEqual(select_fifa().legend[int(len(fifa)/6)],"Top 25th Percentile")

        self.assertEqual(select_fifa().color[int(len(fifa)*5/6)],"#8C2980")
        self.assertEqual(select_fifa().legend[int(len(fifa)*5/6)],"Bottom 25th Percentile")

        self.assertEqual(select_fifa().color[int(len(fifa)*3/4)],"#79D151")
        self.assertEqual(select_fifa().legend[int(len(fifa)*3/4)],"75th Percentile")
        
        
    def test_select_hist_check(self):
        # tests that user selection for skills is valid for histogram
        
        skills3.value = "Defending"        
        self.assertEqual(select_hist()[1],"Defending")
        skills3.value = "Passing"        
        self.assertEqual(select_hist()[1],"Passing")
        skills3.value = "Dribbling"        
        self.assertEqual(select_hist()[1],"Dribbling")
        skills3.value = "Shooting"        
        self.assertEqual(select_hist()[1],"Shooting")         
        skills3.value = "Pace"        
        self.assertEqual(select_hist()[1],"Pace")
        skills3.value = "Overall Rating"        
        self.assertEqual(select_hist()[1],"Overall Rating")
    
    def test_select_fifa2_check(self):
        # checks to ensure 11 players displayed for any formation
        self.assertEqual(len(select_fifa2()),11)
        
        # checks that select_fifa2 includes at least one of each player type
        # regardless of formation
        self.assertIn("Goal Keeper",str(select_fifa2().legend))
        self.assertIn("Defender",str(select_fifa2().legend))
        self.assertIn("Attacker",str(select_fifa2().legend))
        self.assertIn("Midfielder",str(select_fifa2().legend))
        
        # checks to ensure any formation has only one goal keeper
        self.assertEqual(str(select_fifa2().legend).count("Goal Keeper"),1)
    
    # Test that the top 5 countries function
    def testtop5countries(self):
        self.assertEqual(top5countries(), 5)
        
    # Test plotting label for series
    def test_series_label(self):
        numplayers = fifa['Country'].value_counts().sort_values(ascending = False).head(10)
        self.assertEqual(barlabel(numplayers), 10)
    
    # Test plotting label for pandas
    def test_pandas_label(self):
        count_positions = fifa.groupby('Position')['Overall Rating'].count().reset_index(name='Overall Rating').sort_values(['Overall Rating'], ascending=False)                
        self.assertEqual(barlabelpandas(count_positions['Overall Rating']), 17)
        
if __name__ == "__main__":
    unittest.main(argv=['first-arg-is-ignored'], exit=False)

        

Number 1  Team Spain 
                 Name Country Position Group
11            De Gea   Spain    Goal Keeper
19      Sergio Ramos   Spain       Defender
29             Piqué   Spain       Defender
43        Jordi Alba   Spain       Defender
84          Carvajal   Spain       Defender
14   Sergio Busquets   Spain     Midfielder
26       David Silva   Spain     Midfielder
45            Thiago   Spain     Midfielder
66              Isco   Spain     Midfielder
72            Parejo   Spain     Midfielder
105       Iago Aspas   Spain       Attacker 

Number 2  Team France 
                   Name Country Position Group
33         Hugo Lloris  France    Goal Keeper
42     Aymeric Laporte  France       Defender
61       Samuel Umtiti  France       Defender
83      Raphaël Varane  France       Defender
101    Clément Lenglet  France       Defender
15        N'Golo Kanté  France     Midfielder
25          Paul Pogba  France     Midfielder
74      Blaise Matuidi  France     Midfielder
21       

.......................

Number 1  Team Spain 
                 Name Country Position Group
11            De Gea   Spain    Goal Keeper
19      Sergio Ramos   Spain       Defender
29             Piqué   Spain       Defender
43        Jordi Alba   Spain       Defender
84          Carvajal   Spain       Defender
14   Sergio Busquets   Spain     Midfielder
26       David Silva   Spain     Midfielder
45            Thiago   Spain     Midfielder
66              Isco   Spain     Midfielder
72            Parejo   Spain     Midfielder
105       Iago Aspas   Spain       Attacker 

Number 2  Team France 
                   Name Country Position Group
33         Hugo Lloris  France    Goal Keeper
42     Aymeric Laporte  France       Defender
61       Samuel Umtiti  France       Defender
83      Raphaël Varane  France       Defender
101    Clément Lenglet  France       Defender
15        N'Golo Kanté  France     Midfielder
25          Paul Pogba  France     Midfielder
74      Blaise Matuidi  France     Midfielder
21       

.

Number 5  Team Argentina 
                    Name    Country Position Group
386      Gerónimo Rulli  Argentina    Goal Keeper
187      Ezequiel Garay  Argentina       Defender
198    Nicolás Otamendi  Argentina       Defender
306  Nicolás Tagliafico  Argentina       Defender
478      Federico Fazio  Argentina       Defender
23         Paulo Dybala  Argentina     Midfielder
97      Alejandro Gómez  Argentina     Midfielder
190         Éver Banega  Argentina     Midfielder
0          Lionel Messi  Argentina       Attacker
12        Sergio Agüero  Argentina       Attacker
69       Ángel Di María  Argentina       Attacker 




----------------------------------------------------------------------
Ran 24 tests in 13.277s

OK
